# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)


In [2]:
models = Models.from_conf(conf.load('./config/default_tf1x.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default_tf1x.conf'))  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term


INFO:tensorflow:Restoring parameters from ..\models/GIB/2024-07-06_bidding_V2-3114000
INFO:tensorflow:Restoring parameters from ..\models/GIB/2024-07-06_bidding_V2-3114000
INFO:tensorflow:Restoring parameters from ..\models/contract/contract-193200
INFO:tensorflow:Restoring parameters from ..\models/GIB/binfo_V2-1420000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/single_dummy/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\models/playing/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\models/playing/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..

### Running through an example

In [3]:

vuln_ns, vuln_ew = False, False

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# you sit West and hold:
hand = '6.AKJT82.762.K63'

# the auction goes:
auction = ['1D', '3S']
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, seat=2, dealer=0, ddsolver=dds, verbose=False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

Loaded lib dds.dll
DDSolver being loaded - dds mode 1


[{'call': '4H',
  'insta_score': 0.474,
  'adjustment': 24,
  'expected_score': 220,
  'expected_tricks': 10.1,
  'alert': 'False'},
 {'call': 'PASS',
  'insta_score': 0.515,
  'adjustment': 26,
  'expected_score': 83,
  'expected_tricks': 8.9,
  'alert': 'False'}]

In [4]:
auction = ['1D', '3S', '4H', '4S', '5H', 'PASS']
bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.941, 'alert': 'False'}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['J.Q9xx.KQJ8xx.Qx KT8xxxx.x.x.JT9x x.AKJT8x.xxx.Kxx AQ9x.xx.AT9.A8xx 0.77734',
 'x.9xxxx.AKQJTx.9 QT98xxx.x.9x.QJx x.AKJT8x.xxx.Kxx AKJx.Q.8x.AT8xxx 0.76221',
 'Q.Qxxx.KQJT9xx.Q AJT8xxx.9.x.8xxx x.AKJT8x.xxx.Kxx K9xx.xx.A8.AJT9x 0.71338',
 'Jx.Qxxx.AKT9xx.Q QT98xxx.x.J.Jxxx x.AKJT8x.xxx.Kxx AKx.9x.Q8x.AT98x 0.69434',
 'Q.Qxxx.AQTxxx.8x KJT8xxx.x.KJ9.J9 x.AKJT8x.xxx.Kxx A9xx.9x.8.AQTxxx 0.69287',
 'x.Q9xx.AKJT9x.QT KJ98xxx.x.x.J9xx x.AKJT8x.xxx.Kxx AQTx.xx.Q8x.A8xx 0.68408',
 'A.Qxx.KQT8xx.xxx KQ9xxxx.9x.J9.98 x.AKJT8x.xxx.Kxx JT8x.xx.Ax.AQJTx 0.68311',
 'x.Qxxx.AKQ98x.Jx QJT9xxx.9.Tx.Qxx x.AKJT8x.xxx.Kxx AK8x.xx.Jx.AT98x 0.67969',
 'Q.Qxx.AK98xx.9xx AJ98xxx.x.Qx.T8x x.AKJT8x.xxx.Kxx KTxx.9xx.JT.AQJx 0.67578',
 '9.Qxx.AQ98xx.QJ8 QJ8xxxx.x.KTx.xx x.AKJT8x.xxx.Kxx AKTx.9xx.J.AT9xx 0.67090',
 '9.Q9xx.KJT98x.Ax QJT8xxx.xx.A.98x x.AKJT8x.xxx.Kxx AKxx.x.Qxx.QJTxx 0.66748',
 'x.Q9xx.A98xxx.A8 KQT98xx.x.J.JT9x x.AKJT8x.xxx.Kxx AJxx.xx.KQT.Qxxx 0.66309',
 'Q.Qxx.AQJ98x.T8x KJT98xx.9x.K.xxx x.AK

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(['PAD_START','2N','PASS','3C','PASS','3D'], 0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: 0.998
x.AKJT8x.xxx.Kxx AKQ.Qx.AQJ9.Q8xx T8xxx..KT8xxx.JT J9xx.9xxxx..A9xx
x.AKJT8x.xxx.Kxx KQ8.Q9.AKQx.AJTx JTxxx.xx.8xx.Q8x A9xx.xxx.JT9.9xx
x.AKJT8x.xxx.Kxx ATx.Q9x.AKx.AQJx 98xxxx..QJx.T9xx KQJ.xxxx.T98x.8x
x.AKJT8x.xxx.Kxx AKQ.Qxx.AKxx.QTx 98xxx.9x.98.A8xx JTxx.xx.QJTx.J9x
x.AKJT8x.xxx.Kxx AJx.Q9x.AKxx.AQx QT9xx.xx.x.JT8xx K8xx.xx.QJT98.9x
x.AKJT8x.xxx.Kxx KQx.Qx.AKQ8x.ATx T8xx.9.9x.QJ98xx AJ9xx.xxxx.JTx.x
x.AKJT8x.xxx.Kxx AK8.9xx.AKQ9.Axx 9xxxx.x.T8x.T8xx QJTx.Qxx.Jxx.QJ9
x.AKJT8x.xxx.Kxx AKx.Q9.AKT9.A9xx Q8xxx.xx.8x.JT8x JT9x.xxx.QJxx.Qx
x.AKJT8x.xxx.Kxx AK8.xx.AKJ8.AJTx T9xxx.xx.Q9x.Q9x QJxx.Q9x.Txx.8xx
x.AKJT8x.xxx.Kxx AK.Q9x.AK9x.Axxx QJ8xxx.x.Tx.JT8x T9xx.xxx.QJ8x.Q9
x.AKJT8x.xxx.Kxx KJ.Qxx.AKx.AQJ9x QT9xxx.x.JTxx.xx A8xx.9xx.Q98.T8x
x.AKJT8x.xxx.Kxx K8x.Qx.AKQT8.AQx QJxxx.xxx.9x.9xx AT9x.9x.Jxx.JT8x
x.AKJT8x.xxx.Kxx A8x.Q9x.AKQx.AJT JT9xx.xx.8x.Q98x KQxx.xx.JT9x.xxx
x.AKJT8x.xxx.Kxx KQx.Qx.AKQ9x.AJx JT8xx.9x.T8x.Q8x A9xx.xxx.Jx.T9xx
x.AKJT8x.xxx.Kxx AQ8.xx.AKQ9

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [7]:
hands = random_deal().split()
hands # N, E, S, W

['J54.K3.KQT92.985',
 'T9.J98.A8543.T76',
 'AQ73.6542.7.QJ42',
 'K862.AQT7.J6.AK3']

In [8]:
bidder_bots = [BotBid([False, False], hand, models, sampler,i, 0, dds, False) for i, hand in enumerate(hands)]

In [9]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

['PASS', 'PASS', 'PASS', '1N', 'PASS', 'PASS', 'PASS']